In [107]:
import pandas as pd

In [ ]:
df=pd.read_csv("downloads/65k_enriched_topic.csv")

In [332]:
df["len"] = df.text.apply(lambda t: len(t.split(" ")))

In [333]:
#df = df[df["len"] > 7]

In [346]:
import math
def lookup_DOI(val, lookup, key="PMCID"): 
#     print(val, type(val))
#     if val is None or math.isnan(val):
#         return None
#     print(val)
    if type(val) == str:
        #print(val)
        #print(val, type(val))

        match = lookup[key] == val
        DOI = lookup['DOI'][match]
        return DOI.values[0]
    return None

df["pdoi"] = df['PMCID'].apply(lambda x: lookup_DOI(x, pcm, "PMCID"))

 ## TODOs:
 
 * find frequency of domain words within cluster
 * find subfrequency of that word (coocurrence)

# Keyphrase extraction

In [364]:
sentences = list(df["text"])
len(sentences)

56895

In [365]:
from sentence_transformers import SentenceTransformer, util
import time

print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus ... get a coffee in the meantime


Batches:   0%|          | 0/889 [00:00<?, ?it/s]

In [18]:
print("Start clustering")
start_time = time.time()

clusters = util.community_detection(s_embs, threshold=0.4)
print("Clustering took {:.2f} sec".format(time.time() - start_time), "len", len(clusters))

Start clustering


NameError: name 'embeddings' is not defined

In [2]:
!pip install keybert

In [8]:
#df["topic"] = df["topic_name"].apply(lambda terms: " ".join(terms.split(" ")[:2]))

In [377]:
#df.to_csv ("downloads/58k_enriched_topics.csv", index = False, header=True)

## Topic ANalysis

* check KW1 
* check sub KW2s for that KW

* group sentences (group their topics)
* build dictionaries of topics

In [109]:
import pandas as pd
df = pd.read_csv("downloads/57k_octo_clustered.csv")

In [2]:
from sentence_transformers import SentenceTransformer, util
import time

sentences = list(df["text"])
print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
s_embs = model.encode(sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus ... get a coffee in the meantime


Batches:   0%|          | 0/889 [00:00<?, ?it/s]

In [4]:
def pretty_size(size):
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	print("Total size:", total_size)

In [8]:
print("Start clustering")
start_time = time.time()

clusters = util.community_detection(s_embs, threshold=0.85)
print("Clustering took {:.2f} sec".format(time.time() - start_time), "len", len(clusters))

for i, cluster in enumerate(clusters):
    df.loc[cluster, "cluster0.85"] = round(int(i))


Start clustering
Clustering took 9.06 sec len 436


In [11]:
df.to_csv ("downloads/57k_octo_clustered.csv", index = False, header=True)

In [97]:
df[df["cluster0.8"]==1].kw1.unique(), df[df["cluster0.8"]==1].topic.unique()

(array(['existing antibacterial', 'antimicrobial resistance', 'antibiotic',
        'drug resistance', 'resistance antibiotics', 'new antibiotics',
        'antibiotic resistance', 'resistant microbes', 'bacteria arb',
        'antimicrobial drug', 'antibiotics', 'antimicrobial',
        'antibiotics worrisome', 'resistance microorganisms',
        'resistant bacteria', 'antibiotic resistant', 'abuse antibiotics',
        'pneumococcal resistance', 'microbial resistance',
        'resistant pathogens', 'resistance antibiotic',
        'overuse antibiotic', 'bacterial infections',
        'overuse antibiotics', 'antibiotics worldwide',
        'bacteria antibiotics', 'resistance virulence',
        'prevalence antibiotic', 'resistance antimicrobials',
        'multidrug resistance', 'spread antibiotic',
        'bacterial resistance', 'microorganisms antibiotics',
        'antibiotic use', 'antibiotics challenge', 'pathogenic resistance',
        'resistance pseudomonadaceae', 'traditio

In [37]:
#Print for all clusters the top 3 and bottom 3 elements
def print_cl(clusters):
    clus={}
    for i, cluster in enumerate(clusters[:20]):

        print("\nCluster {}, #{} Elements ".format(i, len(cluster)))
        clus["cluster_number " + str(i)] = cluster

        for sentence_id in cluster[0:3]:
            print("\t", sentences[sentence_id])
        print("\t", "...")
        for sentence_id in cluster[-3:]:
            print("\t", sentences[sentence_id])

In [ ]:
from sentence_transformers import SentenceTransformer, util
import time

words = list(set(df["kw1"]))
print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(words, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

In [55]:
!pip install faiss-gpu

     |████████████████████████████████| 85.5 MB 30.8 MB/s            


In [27]:
import faiss

In [23]:
from sentence_transformers import SentenceTransformer, util
import time

words = list(set(df["kw1"])) + list(set(df["topic"]))
print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(words, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus ... get a coffee in the meantime


Batches:   0%|          | 0/489 [00:00<?, ?it/s]

In [41]:
def cluster_x(texts, model=model, threshold=0.85):
    embeddings = model.encode(texts, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
    clusters = util.community_detection(embeddings, threshold=threshold)
    #print("Clustering took {:.2f} sec".format(time.time() - start_time), "len", len(clusters))
    return clusters

In [125]:
df[df["cluster0.6"]==1].index[[1,2,3]]

Int64Index([31, 176, 192], dtype='int64')

In [135]:
from sentence_transformers import SentenceTransformer, util
import time
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def write_child_clusters(cluster_id, col="cluster0.6", thresh=0.8, df=df, write=True):
    texts = list(df[df[col]==cluster_id]["text"])
    idxs = df[df[col]==cluster_id].index
    
    
    cl = cluster_x(texts, model, thresh)
    if write:
        print("cluster ", cluster_id, " has ", len(cl), " child clusters")
        for i,c in enumerate(cl):
            #global dataframe
            df.loc[idxs[c], "c0.6_children"] = str(round(int(cluster_id))) + ":" + str(round(int(i))) #(parent_cluster, child_cluster)
    return cl



In [136]:
import math
for idx in [id for id in set(df["cluster0.6"]) if type(id) == float and not math.isnan(id)]:
    write_child_clusters(idx)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

cluster  1.0  has  13  child clusters


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

cluster  7.0  has  3  child clusters


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

cluster  6.0  has  5  child clusters


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

cluster  31.0  has  2  child clusters


Batches:   0%|          | 0/18 [00:00<?, ?it/s]

cluster  0.0  has  5  child clusters


In [115]:
#combine all keywords
#pick one with highest count

In [ ]:
def summary_term(kws):
    #count keyterms
    #use max val as representative

In [140]:
df["c0.6_children"].unique()

array(['0.0:1', '394.0:0', '7:1', '47:2', '0.0:0', '327:0', '31.0:0',
       '24:0', '31.0:1', '371.0:0', '0.0:3', '20:0', '225.0:0', '178.0:0',
       '1:6', '177.0:0', '31:1', '15:1', '0.0:4', '0:1', '3:0', '0.0:2',
       '283:0', '32.0:0', '6.0:0', '107:1', '4:0', '16:0', '18:0',
       '32.0:1', '524:0', '6.0:3', '5:1', '45:1', '10:1', '255:0',
       '11.0:0', '14:0', '217:0', '2:0', '11.0:1', '144:0', '63:2',
       '49:2', '23.0:0', '29:0', '0:0', '14.0:0', '13.0:0', '14.0:1',
       '1:0', '164:0', '6.0:4', '34:0', '6.0:1', '19:0', '445:0', '308:0',
       '2.0:3', '12.0:0', '2.0:4', '21:1', '6.0:2', '2.0:0', '256:0',
       '156:0', '2:2', '21:0', '5:0', '232:0', '2.0:5', '2.0:1', '68:0',
       '10.0:2', '2.0:2', '17:0', '10.0:1', '28:0', '10.0:3', '245:0',
       '165:0', '5.0:4', '3:1', '222:0', '55:0', '8:1', '54:1', '5.0:0',
       '2:5', '42:0', '137:0', '2.0:6', '5.0:2', '102:0', '5.0:1',
       '261:0', '3:2', '4.0:0', '15:3', '6:0', '10:0', '17:1', '3.0:1',
       '3

In [97]:
for c_id in set(df["cluster0.6"])write_child_clusters(0, write=False))

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

5

In [178]:
len(tuple(int(float(x)) for x in "24:0".split(":")))

2

In [169]:
df["c0.6_children"] = df["c0.6_children"].apply(lambda code: tuple(int(float(x)) for x in code.split(":")) if type(code) == str else code)

In [174]:
#df[df["cluster0.6"].isna()]

In [185]:
from collections import Counter

def cluster_kw_frequencies(df, topic_name, target, col='kw1'):
    d = df[df[target] == topic_name]
    l = list(d[col])
    return Counter(l)



In [175]:
cluster_kw_frequencies(df,(24,0)).most_common(1)[0][0]

'chronic kidney'

In [176]:
df.to_csv ("downloads/data.csv", index = False, header=True)

In [189]:
cluster_kw_frequencies(df,1, "cluster0.6")

Counter({'zoonotic bacteria': 1,
         'resistance acinetobacter': 1,
         'existing antibacterial': 1,
         'potent antibacterial': 1,
         'antimicrobial therapeutics': 1,
         'foodborne pathogens': 1,
         'antimicrobial resistance': 30,
         'wound infections': 2,
         'lactmase esbl': 1,
         'streptococcus pneumoniae': 15,
         'enterococcus': 4,
         'lactamase': 1,
         'antibiotic': 32,
         'sepsis': 5,
         'bacterial infection': 2,
         'beta lactamase': 1,
         'prevention': 1,
         'lactam antibiotics': 3,
         'respiratory infections': 1,
         'infections hai': 1,
         'pathogens': 9,
         'drug resistance': 12,
         'bacterial': 5,
         'resistant pneumococcus': 1,
         'contamination touch': 1,
         'biofilms cause': 2,
         'resistance antibiotics': 12,
         'pseudomonas': 16,
         'diarrhea caused': 1,
         'pneumococcal meningitis': 1,
         'intens

In [181]:
df["children_term"] = df["c0.6_children"].apply(lambda x: cluster_kw_frequencies(df,x).most_common(1)[0][0] if( type(x) == tuple and len(x) == 2) else x)

In [192]:
df["parent_term"] = df["cluster0.6"].apply(lambda x: cluster_kw_frequencies(df,x, "cluster0.6").most_common(1)[0][0] if (type(x) == float and not math.isnan(x)) else x)

In [194]:
df.to_csv("downloads/data.csv", index = False, header=True)

In [229]:
pt = set(x for x in list(df["parent_term"]) if type(x) == str)
topics = set(x for x in list(df["topic"]) if type(x) == str)

In [221]:
def get_children(parent_term):
    return list(set(x for x in df[df["parent_term"] == parent_term  ]["children_term"] if type(x) == str))

,title,article_link,text,source,PMID,DOI,coreId,oai,kws,kw1,domain_words,topic,cluster0.4,cluster0.6,cluster0.8,cluster0.85,c0.6_children,children_term,parent_term
2,Protein Synthesis Attenuation by Phosphorylati...,NaN,Chagas' disease is a potentially life-threaten...,core_pubmed,NaN,NaN,8644680.0,oai:pubmedcentral.nih.gov:3218062,"[('chagas disease', 0.8), ('parasite trypanoso...",chagas disease,"[""Chagas'"", 'life-threatening', 'unicellular',...",chagas cruzi,0.0,7.0,16.0,17.0,"(7, 1)",chagas disease,leishmaniasis
4,Role of immune cells in pancreatic cancer from...,NaN,Pancreatic cancer (PC) remains difficult to tr...,labels_oct7,27930550.0,10.1097/MD.0000000000005541,NaN,NaN,"[('pancreatic cancer', 0.7705), ('pancreatic',...",pancreatic,"['Pancreatic', '(PC)', 'treat,', 'anticancer']",pancreatic adenocarcinoma,NaN,NaN,43.0,46.0,"(0, 0)",obesity,NaN
7,Analysis of peptidoglycan precursors in vancom...,NaN,Vancomycin resistance in enterococci is an inc...,core_pubmed,NaN,10.1042/bj3010005,3671542.0,oai:pubmedcentral.nih.gov:1137133,"[('resistance enterococci', 0.7862), ('vancomy...",resistance enterococci,"['Vancomycin', 'enterococci', 'problem,', 'phe...",enterococci vancomycin,NaN,NaN,172.0,211.0,"(0, 0)",obesity,NaN
11,Burden of Proteinuria and Risk Factors of Chro...,NaN,"In the recent times , Chronic Kidney Diseases ...",a {negative_adjectives} ... {problem_nouns},28969164.0,10.7860/jcdr/2017/24492.10430,NaN,NaN,"[('chronic kidney', 0.6965), ('kidney diseases...",chronic kidney,"['In', ',', 'Chronic', 'Kidney', 'Diseases', '...",kidney renal,NaN,24.0,40.0,39.0,"(24, 0)",chronic kidney,chronic kidney
14,Coronavirus-positive Nasopharyngeal Aspirate a...,NaN,Severe acute respiratory syndrome (SARS) has c...,core_pubmed,NaN,NaN,8524187.0,oai:pubmedcentral.nih.gov:3035547,"[('syndrome sars', 0.6762), ('sars caused', 0....",syndrome sars,"['Severe', '(SARS)', 'worldwide.']",sars coronavirus,0.0,NaN,23.0,25.0,"(371, 0)",dupuytren,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56850,Pleomorphic Adenoma (Benign Mixed Tumour) of t...,NaN,Pleomorphic adenoma is the benign tumor of sal...,core_pubmed,NaN,NaN,8661112.0,oai:pubmedcentral.nih.gov:3244097,"[('pleomorphic adenoma', 0.8262), ('adenoma', ...",adenoma,"['Pleomorphic', 'adenoma', 'salivary', 'glands...",odontogenic adenomatoid,NaN,NaN,413.0,410.0,NaN,NaN,NaN
56859,Evolution of drug resistance in Candida albicans.,NaN,With the limited availability of antifungal dr...,a {negative_adjectives} ... {problem_nouns},12142485.0,10.1146/annurev.micro.56.012302.160907,NaN,NaN,"[('resistance fungal', 0.7305), ('fungal patho...",resistance fungal,"['With', 'antifungal', ',', '.']",congenital surgery,0.0,546.0,244.0,290.0,NaN,NaN,azole resistance
56861,Effectiveness of a Therapeutic Tai Ji Quan Int...,NaN,Falls in older adults are a serious public hea...,a {negative_adjectives} ... {problem_nouns},30208396.0,10.1001/jamainternmed.2018.3915,NaN,NaN,"[('falls older', 0.6789), ('falls', 0.5027), (...",falls,"['Falls', '.']",falls falling,NaN,14.0,34.0,32.0,"(14, 0)",falls,osteoporosis
56865,The Role of Anti-PD-1/PD-L1 in the Treatment o...,NaN,Skin cancers remain the most common group of c...,pm_4kpos,32447657.0,10.1007/s40259-020-00428-9,NaN,NaN,"[('skin cancers', 0.7199), ('cancers globally'...",skin cancers,"['Skin', 'globally,', 'rise.']",melanoma melasma,NaN,2.0,348.0,265.0,"(2, 6)",skin cancer,lung cancer


In [236]:
tree2 = {}
for t in topics:
    tree2[t] = list(set(df[df["topic"] == t].kw1))

In [248]:
import json
with open("downloads/topic_hier_v0.json", "w") as f:
    f.write(json.dumps(tree2))

In [249]:
tree2

{'mouth hfmd': ['species tropism',
  'pertussis',
  'endemic rubella',
  'disease pigs',
  'virulent em',
  'foodborne outbreaks',
  'nepal concern',
  'diseases',
  'disease',
  'worldwide pertussis',
  'animal viruses',
  'severe immune',
  'hfrs health',
  'measles vaccines',
  'prevalence virus',
  'disease fmd',
  'cells virus',
  'modeling infectious',
  'threat biodiversity',
  'blv',
  'hfmd outbreaks',
  'coxsackievirus a16',
  'canine nasal',
  'virus fmdv',
  'zoonoses',
  'diseases paramount',
  'animals transit',
  'chronic fever',
  'mouth disease',
  'endemic zoonoses',
  'cowpox severe',
  'disease cwd',
  'pestiviruses',
  'disease hfmd',
  'prevalence infected',
  'fever',
  'ev71 infection',
  'enterovirus',
  'tibetan pigs',
  'vtec isolates',
  'cooperiiasis',
  'contagious nature',
  'study hfrs',
  'wildlife diseases',
  'chickenpox',
  'fever mcf',
  'virus foot',
  'pastern dermatitis',
  'slovakia fever',
  'dangerous infections'],
 'pesticides pesticide': ['p

In [217]:
set() or {}

{}

In [28]:
import numpy as np
emb = embeddings.cpu()
emb = emb.numpy()
d = emb.shape[1]
emb.shape, len(df)
#np.array(df["emb"][1])[:].shape
x = [list(e) for e in emb ]
#df["emb"] = x

In [29]:

def create_index(vectors):
    faiss_index = faiss.IndexFlatL2(len(vectors[0]))
    faiss_index.add(vectors)

#     print(faiss_index.ntotal)

    return faiss_index

#export
def query_index(text, embedder, target_list, index, with_distance=False, k=10):
    embedding = embedder.encode([text])
    distances, indices = index.search(embedding, k)
    if with_distance:
        return [(target_list[index], distances[0][i]) for i, index in enumerate(indices[0])]
    return [target_list[i] for i in indices[0]]

In [30]:
index = create_index(emb)

In [93]:
#kws = set(list(df["kw2"]))

In [31]:
stems = {}

for kw in [kw for kw in set(list(df["kw1"])) if type(kw) == str]:
    stemmed = " ".join([Slice(k, 3).output for k in kw.split(" ")])
    if stems.get(stemmed):
        stems[stemmed] += [kw]
    else:
        stems[stemmed] = [kw]

In [32]:
len(df.kw1.unique()), len(df.topic.unique())

(30169, 1124)

In [33]:
df[df["cluster0.85"] == 7]

,title,article_link,text,source,PMID,DOI,coreId,oai,kws,kw1,domain_words,topic,cluster0.4,cluster0.6,cluster0.8,cluster0.85
143,The SNP rs931794 in 15q25.1 Is Associated with...,NaN,Lung cancer is one of the most common human ma...,pm_4kpos,26079375.0,10.1371/journal.pone.0128201,NaN,NaN,"[('lung cancer', 0.8153), ('cancer common', 0....",lung cancer,"['Lung', 'worldwide.']",lung radon,3.0,2.0,6.0,7.0
198,A phase I evaluation of oral CI-1033 in combin...,NaN,"In the United States, lung cancer represents t...",pm_4kpos,17409987.0,10.1016/s1556-0864(15)31635-x,NaN,NaN,"[('lung cancer', 0.7447), ('common cancer', 0....",lung cancer,"['In', 'United', 'States,', 'cancer,', 'cancer...",lung radon,3.0,2.0,6.0,7.0
200,Assessing the effectiveness and safety of lipo...,NaN,Lung cancer is still the leading cause of canc...,pm_4kpos,23413951.0,10.1186/1745-6215-14-45,NaN,NaN,"[('lung cancer', 0.7285), ('mortality worldwid...",lung cancer,"['Lung', 'cancer-related', 'worldwide.']",lung radon,3.0,2.0,6.0,7.0
275,Whole genome microarray analysis in non-small ...,NaN,"Lung cancer is a serious health problem , sinc...",a {negative_adjectives} ... {problem_nouns},26019623.0,10.1080/13102818.2014.989179,NaN,NaN,"[('lung cancer', 0.7225), ('cancer health', 0....",lung cancer,"['Lung', ',', '.']",lung radon,3.0,2.0,6.0,7.0
1121,Capsaicin inhibits benzo(a)pyrene-induced lung...,NaN,Lung cancer is a serious health problem in mos...,a {negative_adjectives} ... {problem_nouns},22735861.0,10.1007/s00011-012-0511-1,NaN,NaN,"[('lung cancer', 0.693), ('cancer health', 0.5...",lung cancer,"['Lung', 'profusely', '.']",lung radon,3.0,2.0,6.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55362,Selection of Patients With Non-Small-Cell Lung...,NaN,Cancer of the lung is rapidly increasing in in...,core_pubmed,NaN,NaN,3822412.0,oai:pubmedcentral.nih.gov:1306429,"[('cancer lung', 0.6125), ('breast cancer', 0....",cancer lung,"['Cancer', 'overtake', 'women.']",lung death,3.0,2.0,6.0,7.0
55718,NaN,NaN,Lung cancer is a fatal disease and a serious h...,a {negative_adjectives} ... {problem_nouns},33059701.0,10.1186/s13014-020-01682-5,NaN,NaN,"[('lung cancer', 0.7627), ('cancer fatal', 0.5...",lung cancer,"['Lung', '.']",lung death,3.0,2.0,6.0,7.0
55837,Treatment of advanced non small cell lung cancer,NaN,Lung cancer is the major cause of cancer death...,core_pubmed,NaN,NaN,8668964.0,oai:pubmedcentral.nih.gov:3256511,"[('lung cancer', 0.7483), ('cancer death', 0.7...",lung cancer,"['Lung', 'world.']",lung death,3.0,2.0,6.0,7.0
55987,Smoking Cessation: An Integral Part of Lung Ca...,NaN,Lung cancer is the leading cause of cancer dea...,core_pubmed,NaN,NaN,8456346.0,oai:pubmedcentral.nih.gov:2945268,"[('lung cancer', 0.7485), ('cancer death', 0.6...",lung cancer,"['Lung', 'US.']",lung death,3.0,2.0,6.0,7.0


In [36]:
query_index("lung damage", model, words, index, True, 200)

[('lung damage', 6.127926e-13),
 ('lung injury', 0.16892102),
 ('lung injuries', 0.19233173),
 ('pulmonary injury', 0.2962226),
 ('lung toxicity', 0.44744468),
 ('lung death', 0.47276753),
 ('pulmonary contusion', 0.49533704),
 ('pulmonary toxicity', 0.50963324),
 ('lung lesions', 0.5407748),
 ('lung disease', 0.5478273),
 ('lung cancer', 0.554504),
 ('patients lung', 0.5709009),
 ('lung', 0.5730413),
 ('respiratory failure', 0.5748451),
 ('lung surgery', 0.59106314),
 ('cause lung', 0.59256303),
 ('pulmonary lesion', 0.6029865),
 ('chest injuries', 0.6139743),
 ('acute lung', 0.62378794),
 ('lung diseases', 0.62529266),
 ('cancer lung', 0.626169),
 ('chest injury', 0.6340926),
 ('respiratory complications', 0.6480157),
 ('airway trauma', 0.6559905),
 ('liver damage', 0.6590115),
 ('lung immaturity', 0.66427356),
 ('tissue damage', 0.6718446),
 ('chest trauma', 0.6733145),
 ('pulmonary collapse', 0.6748392),
 ('pulmonary hemorrhage', 0.6846057),
 ('lung cancers', 0.68955356),
 ('kidney

In [16]:
stems

{'opioid addictio': ['opioid addiction'],
 'sympatheti dystrophy': ['sympathetic dystrophy'],
 'materna inflamma': ['maternal inflammation'],
 'balance dysfunction': ['balance dysfunction'],
 'hepcidi': ['hepcidin'],
 'glenohu': ['glenohumeral'],
 'avian patholo': ['avian pathology'],
 'solid tumor': ['solid tumor'],
 'empowe': ['empowerment'],
 'subjects parkinso': ['subjects parkinson'],
 'safety': ['safety'],
 'infectio blood': ['infection blood'],
 'aortic dissectio': ['aortic dissections', 'aortic dissection'],
 'genome medici': ['genome medicine'],
 'path carcino': ['path carcinogenesis'],
 'transplanta neurode': ['transplantation neurodegenerative'],
 'distrust health': ['distrust health'],
 'transplant lymphoproli': ['transplant lymphoproliferative'],
 'mdr aerugi': ['mdr aeruginosa'],
 'smokers': ['smokers'],
 'target inhibi': ['target inhibitors'],
 'microbia pathoge': ['microbial pathogenesis', 'microbial pathogens'],
 'sahel transitio': ['sahel transition'],
 'menisca sizin

In [128]:
def cluster_kw_frequencies(df, topic_name, col='kw1'):
    d = df[df["tn"] == topic_name]
    l = list(d[col])
    return Counter(l)

cluster_kw_frequencies(df, "streptococcus streptococci")

Counter({'zoonotic bacteria': 1,
         'streptococcus': 5,
         'resistance streptococcus': 1,
         'pharyngitis gabhs': 1,
         'pathogen fish': 1,
         'streptococcal prevalence': 1,
         'rheumatic fever': 2,
         'streptococcus gas': 3,
         'phlegmonous gastritis': 1,
         'streptococci ggs': 1,
         'aggregatibacter': 1,
         'strep': 1,
         'lugdunensis endocarditis': 1,
         'streptococcus milleri': 1,
         'streptococcus equisimilis': 1,
         'staphylococcus lugdunensis': 1,
         'group streptococci': 1,
         'streptococci gbs': 1,
         'streptococci scrogroups': 1,
         'streptococcus gbs': 1})

In [114]:
stems

{'space heating': ['space heating'],
 'heamodia treatment': ['heamodialysis treatment'],
 'thalasse constitu': ['thalassemia constitutes', 'thalassemia constitute'],
 'zinc': ['zinc'],
 'fragile infants': ['fragile infants'],
 'doping abuse': ['doping abuse'],
 'klebsiella species': ['klebsiella species'],
 'morning pills': ['morning pills'],
 'conside pharmaco': ['consider pharmacokinetic'],
 'absorptio grafted': ['absorption grafted'],
 'existe salmone': ['existence salmonella'],
 'perfora head': ['perforation head'],
 'edcs': ['edcs'],
 'nemali myopathy': ['nemaline myopathy'],
 'results hbv': ['results hbv'],
 'priority': ['priority'],
 'thorotra granula': ['thorotrast granulations'],
 'drug use': ['drug use'],
 'epide periodi': ['epidemics periodic'],
 'drawback flux': ['drawback flux'],
 'used journals': ['used journals'],
 'pathoge nairovi': ['pathogen nairovirus'],
 'adole public': ['adolescents public', 'adolescence public'],
 'syndrome srus': ['syndrome srus'],
 'permane fill

In [89]:
query_index("chaga disease", model, words, index, True, 50)

[('chagas disease', 0.29422185),
 ('causes chagas', 0.65663195),
 ('disease cgd', 0.75332314),
 ('syndrome cdgs', 0.88411117),
 ('cruzi disease', 0.89329267),
 ('disease african', 0.89801735),
 ('tropical disease', 0.90774506),
 ('disease cacao', 0.9088952),
 ('disease cjd', 0.91397667),
 ('disease cad', 0.9200283),
 ('disease igcd', 0.9300076),
 ('disease southwestern', 0.9422728),
 ('enigmatic disease', 0.9456077),
 ('barlow disease', 0.953141),
 ('discovery chagas', 0.95525414),
 ('tropical diseases', 0.9595732),
 ('maculopathy', 0.96156895),
 ('diseases tropical', 0.9628924),
 ('chlorosis', 0.9706787),
 ('skin disease', 0.9709861),
 ('disease considered', 0.9753209),
 ('fujimoto disease', 0.9793736),
 ('aaa disease', 0.9814501),
 ('chondromalacia', 0.98326993),
 ('body disease', 0.9843182),
 ('calciphylaxis', 0.9843546),
 ('dermatological disease', 0.98579615),
 ('human disease', 0.98724216),
 ('chiasmal syndrome', 0.9886662),
 ('dermatosis', 0.9903994),
 ('genetic disease', 0.9911

In [42]:
cluster_kw_frequencies(df, "chagas cruzi trypanosoma" )

Counter({'chagas disease': 3,
         'visceral leishmaniosis': 2,
         'cutaneous leishmaniasis': 8,
         'leishmaniasis vl': 14,
         'leishmaniasis': 22,
         'canine leishmaniasis': 2,
         'sleeping sickness': 2,
         'leishmania infantum': 1,
         'american trypanosomiasis': 1,
         'trypanosoma cruzi': 4,
         'trypanosomiasis hat': 3,
         'trypanosomiasis chagas': 2,
         'african trypanosomiasis': 10,
         'visceral leishmaniasis': 6,
         'trypanosoma': 2,
         'trypanosomatid mitochondria': 1,
         'canine leishmaniosis': 1,
         'leishmaniasis considered': 1,
         'carrier leishmaniasis': 1,
         'africa trypanosomiasis': 1,
         'infection leishmania': 1,
         'trypanosoniasis chagas': 1,
         'trypanosomes': 1,
         'leishmaniases': 2,
         'leishmania': 2,
         'leishmaniasis brazilian': 1,
         'trypanosomosis': 1,
         'africa trypanosomosis': 1,
         'leishman

In [14]:
class Slice(object):
    def __init__(self, root, leng):
        self.root = root # The original word
        self.leng = leng # How many morphemes should be used as the prefix for the new portmanteau
        self.morphemes = []
        self.output = '' #
        self.slice()

    def slice(self):
        import re
        ex = r'([^aeiou]*[aeiou]*)|[aeiou]*[^aeiou]*[aeiou]*'
        #ex = r'[^aeiou]*[aeiou]*[^aeiou]*|[aeiou]*[^aeiou]'
        root = self.root

        # Full list of morphemes for future use
        while root != '':
            end = re.match(ex, root).end()
            self.morphemes.append(root[0:end])
            root = root[end:]

        # Check that the number given isnt more than is available
        if len(self.morphemes) < self.leng:
            self.leng = len(self.morphemes)

        #Stitch together the output word
        g = 0
        while g < self.leng :
            self.output += self.morphemes[g]
            g += 1

In [212]:
for cluster_idxs in clusters[:1]:
    sample = df.iloc[cluster_idxs]
    sample

32541

In [214]:
#Print for all clusters the top 3 and bottom 3 elements
clus = {}
for i, cluster in enumerate(clusters[:20]):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    clus["cluster_number " + str(i)] = cluster
    
    for sentence_id in cluster[0:3]:
        print("\t", sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", sentences[sentence_id])


Cluster 1, #241 Elements 
	 Obesity represents a serious health problem as it is rapidly increasing worldwide .
	 Obesity is a serious and rapidly growing health problem worldwide .
	 Obesity is a serious public health problem that is growing alarmingly worldwide .
	 ...
	 Morbid obesity and its rapidly increasing prevalence has became a serious health , social and economic problem not only in Europe , but in other developed countries in the Western world .
	 Obesity is a serious health problem for all age groups and it also promotes Type 2 diabetes , cardiovascular diseases , cancer and hypertension .
	 Obesity is a serious health problem that increased risk for many complications , including diabetes and cardiovascular disease .

Cluster 2, #118 Elements 
	 Hepatitis C virus ( HCV ) is a serious global health problem and establishes chronic infection in a significant number of infected humans worldwide .
	 Hepatitis C virus ( HCV ) is a serious global health problem affecting approx

# NetworkX hierarchy

In [6]:
dd = df.copy()

In [17]:
dd["problem_id"] = [*range(0, len(dd))]

In [18]:
dd

,title,article_link,text,source,PMID,DOI,Create Date,coreId,oai,topic_name,topic_id,kws,kw1,kw2,domain_words,problem_id
0,Dextran Fractional Clearance Studies in Acute ...,NaN,Dengue is a potentially serious common viral i...,core_pubmed,NaN,NaN,NaN,8608925.0,oai:pubmedcentral.nih.gov:3160290,dengue mosquito fever,22,"[('dengue potentially', 0.7544), ('dengue', 0....",dengue,viral infection,"['Dengue', 'treatment.']",0
1,[New insight into bacterial zoonotic pathogens...,NaN,The article also includes new emerging zoonoti...,a {negative_adjectives} ... {problem_nouns},25902698.0,NaN,NaN,NaN,NaN,streptococcus streptococci pharyngitis,91,"[('zoonotic bacteria', 0.6903), ('emerging zoo...",zoonotic bacteria,staphylococcus intermedius,"['The', 'zoonotic', ',', 'ECDC', ':', 'Strepto...",1
2,Protein Synthesis Attenuation by Phosphorylati...,NaN,Chagas' disease is a potentially life-threaten...,core_pubmed,NaN,NaN,NaN,8644680.0,oai:pubmedcentral.nih.gov:3218062,chagas cruzi trypanosoma,40,"[('chagas disease', 0.8), ('parasite trypanoso...",chagas disease,trypanosoma,"[""Chagas'"", 'life-threatening', 'unicellular',...",2
3,Mapping a gene for adult-onset primary open-an...,NaN,Glaucoma is the third-leading cause of blindne...,core_pubmed,NaN,NaN,NaN,7466653.0,oai:pubmedcentral.nih.gov:1712411,retinal glaucoma blindness,348,"[('cause blindness', 0.7102), ('glaucoma', 0.7...",glaucoma,blindness,"['Glaucoma', 'third-leading', 'world,', '>13.']",3
4,Role of immune cells in pancreatic cancer from...,NaN,Pancreatic cancer (PC) remains difficult to tr...,labels_oct7,27930550.0,10.1097/MD.0000000000005541,NaN,NaN,NaN,pancreatic adenocarcinoma pdac,34,"[('pancreatic cancer', 0.7705), ('pancreatic',...",pancreatic,anticancer therapies,"['Pancreatic', '(PC)', 'treat,', 'anticancer']",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56890,Biological monitoring today and tomorrow.,NaN,The biggest problem with most biological monit...,limitation|problem ... with ... l=be that,7846491.0,NaN,NaN,NaN,NaN,congenital surgery autosomal,-1,"[('biological monitoring', 0.5138), ('biologic...",biological,problem biological,"['The', '.']",56890
56891,Rodent model of reproductive tract leiomyomata...,NaN,Uterine myometrial tumors are the most commonl...,core_pubmed,NaN,NaN,NaN,7604771.0,oai:pubmedcentral.nih.gov:1870894,endometrial endometriosis gynecological,173,"[('myometrial tumors', 0.8078), ('gynecologica...",myometrial tumors,neoplasm women,"['Uterine', 'myometrial', 'gynecological', 'wo...",56891
56892,Unexpected Death or Suicide by a Child or Adol...,NaN,The death of a young patient is a difficult bu...,core_pubmed,NaN,NaN,NaN,8661200.0,oai:pubmedcentral.nih.gov:3244296,dying terminally terminal,414,"[('young patient', 0.5394), ('death', 0.509), ...",death,death young,"['The', 'medicine.']",56892
56893,RNAi for Plant Functional Genomics,NaN,A major challenge in the post-genome era of pl...,core_pubmed,NaN,NaN,NaN,8108330.0,oai:pubmedcentral.nih.gov:2447448,plant arabidopsis dormancy,163,"[('plant genome', 0.7875), ('genes plant', 0.6...",plant genome,plant biology,"['A', 'post-genome', 'genome.']",56893


In [19]:
import networkx as nx

dd = dd.reset_index(drop=True)
samp = dd#.sample(frac=0.2)

def graph(df, meta_df):   
    G = nx.from_pandas_edgelist(df, 'kw2', 'kw1', create_using=nx.DiGraph()) #node_attr
    node_attr = meta_df.set_index("problem_id").to_dict('index')
    nx.set_node_attributes(G, node_attr)
    return G

In [24]:
G= graph(dd, dd)

In [25]:
nx.write_gexf(G, "downloads/" + ".gexf")

In [27]:
from networkx.algorithms.clique import find_cliques, find_cliques_recursive
from networkx.algorithms.cluster import generalized_degree, triangles
cl = find_cliques_recursive(G)
#gd = generalized_degree(G)

In [34]:
from networkx.algorithms import community

communities_generator = community.girvan_newman(G)

In [37]:
!pip install pyvis

In [39]:
from pyvis.network import Network
nt = Network('1000px', '1000px')
# populates the nodes and edges data structures
nt.from_nx(G)
nt.show('G.html')

AssertionError: 

In [3]:
import pandas as pd

df = pd.read_csv("downloads/57k_enriched_topics_kw.csv")

In [31]:
with open("vocab30k.txt") as f:
    vocab30k = f.read().split("\t\n")

def extract_domain_words(sent, vocab=vocab30k):
    words = sent.split(" ")
    domain = []
    for word in words:
        if word not in vocab:
            domain += [word]
    return domain

df["domain_words"] = df["text"].apply(extract_domain_words)

In [32]:
#df.to_csv ("downloads/57k_enriched_topics_kw.csv", index = False, header=True)

In [5]:
df[df["topic_name"] == "chagas cruzi trypanosoma"]

,title,article_link,text,source,PMID,DOI,Create Date,coreId,oai,topic_name,topic_id,kws,kw1,kw2,domain_words
2,Protein Synthesis Attenuation by Phosphorylati...,NaN,Chagas' disease is a potentially life-threaten...,core_pubmed,NaN,NaN,NaN,8644680.0,oai:pubmedcentral.nih.gov:3218062,chagas cruzi trypanosoma,40,"[('chagas disease', 0.8), ('parasite trypanoso...",chagas disease,trypanosoma,"[""Chagas'"", 'life-threatening', 'unicellular',..."
295,Mechanisms of resistance and susceptibility to...,NaN,Several animal models have been established to...,core_pubmed,NaN,NaN,NaN,8536862.0,oai:pubmedcentral.nih.gov:3052183,chagas cruzi trypanosoma,10,"[('visceral leishmaniosis', 0.6723), ('leishma...",visceral leishmaniosis,leishmaniosis,"['Several', 'visceral', 'leishmaniosis', '(VL)..."
613,Risk factors and prediction analysis of cutane...,NaN,Cutaneous leishmaniasis is currently a serious...,a {negative_adjectives} ... {problem_nouns},29578084.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('cutaneous leishmaniasis', 0.8051), ('leishm...",cutaneous leishmaniasis,leishmaniasis,"['Cutaneous', 'leishmaniasis', 'Africa', ',', ..."
640,New antigens for the serological diagnosis of ...,NaN,Visceral leishmaniasis ( VL ) still represents...,a {negative_adjectives} ... {problem_nouns},30571783.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('leishmaniasis vl', 0.7777), ('visceral leis...",leishmaniasis vl,leishmaniasis,"['Visceral', 'leishmaniasis', '(', 'VL', ')', ..."
893,The potential of live attenuated vaccines agai...,NaN,Cutaneous Leishmaniasis is a serious public he...,a {negative_adjectives} ... {problem_nouns},32027892.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('cutaneous leishmaniasis', 0.8306), ('leishm...",cutaneous leishmaniasis,leishmaniasis,"['Cutaneous', 'Leishmaniasis', ',', '.']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16810,Human African trypanosomiasis of the CNS: curr...,NaN,"Human African trypanosomiasis (HAT), also know...",core_pubmed,NaN,10.1172/JCI200421052,NaN,3408928.0,oai:pubmedcentral.nih.gov:338269,chagas cruzi trypanosoma,39,"[('trypanosomiasis hat', 0.7577), ('african tr...",trypanosomiasis hat,trypanosomiasis,"['Human', 'African', 'trypanosomiasis', '(HAT)..."
16971,A validated,NaN,Leishmaniasis is a serious health problem that...,a {negative_adjectives} ... {problem_nouns},33293176.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('leishmaniasis health', 0.7843), ('leishmani...",leishmaniasis,suitable vaccine,"['Leishmaniasis', '.']"
17087,Dispersion of Lutzomyia longipalpis and expans...,NaN,"In Brazil , visceral leishmaniasis ( VL ) is a...",a {negative_adjectives} ... {problem_nouns},30201037.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('leishmaniasis vl', 0.8235), ('visceral leis...",leishmaniasis vl,leishmaniasis,"['In', 'Brazil', ',', 'visceral', 'leishmanias..."
17094,"Use of crude, FML and rK39 antigens in ELISA t...",NaN,Visceral leishmaniasis is a disease caused by ...,a {negative_adjectives} ... {problem_nouns},21195550.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('visceral leishmaniasis', 0.8356), ('leishma...",visceral leishmaniasis,leishmaniasis,"['Visceral', 'leishmaniasis', 'Leishmania', '(..."


In [3]:
from keybert import KeyBERT
kw_model = KeyBERT()

def get_kw(text, kw_model=kw_model):
    kws = kw_model.extract_keywords(text, 
                          keyphrase_ngram_range=(1, 2), 
                          stop_words='english',
                          use_mmr=True, 
                          diversity=0.2)
    
    kws.sort(key=lambda x: x[1], reverse=True)
    return kws

In [19]:
#df["kws"] = None

In [24]:
step=10000
for i in range(0, len(df), step):
    print(i)
    df.loc[i:i+step, "kws"] = df.loc[i:i+step, "text"].apply(get_kw)

0
10000
20000
30000
40000
50000


In [250]:
df.to_csv("downloads/data.csv", index = False, header=True)

In [25]:
def likely(ts):
    #if a unigram is in the two highest likely probs THEN take the unigram]
    try:
        s0=ts[0][0]
        s1=ts[1][0]
        if len(s0.split(" ")) > 1 and len(s1.split(" ")) == 1:
            return s1
        return s0
    except:
        return None


In [26]:
def contender(topics):
    #take the first "different" option that breaks with the main options
    #iterate until the topic that breaks with the terms of previous topics
    terms = []
    idx=0
    for idx, t in enumerate(topics):
        words = t[0].split(" ")
#         print(words, "words")
        if len(set.intersection(set(words), set(terms))) < 1:
            break #word
        terms += words
    return likely(topics[idx+1:])

In [27]:
df["kw1"] = df.kws.apply(likely)

In [28]:
df["kw2"] = df.kws.apply(contender)